In [1]:
from sklearn.linear_model import Lasso 

import pandas as pd
from readii.io.loaders import loadImageDatasetConfig, loadFileToDataFrame
from pathlib import Path
import numpy as np

In [2]:
# Load in the configuration file
config = loadImageDatasetConfig("NSCLC-Radiomics", Path("../../config/datasets"))

# Initialize dataset parameters
CLINICAL_DATA_FILE = config["CLINICAL_FILE"]
DATASET_NAME = config["DATA_SOURCE"] + "_" + config["DATASET_NAME"]

RANDOM_SEED = 10

# general data directory path setup
DATA_DIR_PATH = Path("../../data")
RAW_DATA_PATH = DATA_DIR_PATH / "rawdata" / DATASET_NAME
PROC_DATA_PATH = DATA_DIR_PATH / "procdata" / DATASET_NAME
RESULTS_DATA_PATH = DATA_DIR_PATH / "results" / DATASET_NAME

pyradiomics_settings = "pyradiomics_original_all_features"

In [3]:
# Load clinical data file
clinical_data = loadFileToDataFrame((RAW_DATA_PATH / "clinical" / CLINICAL_DATA_FILE))

In [15]:
# Load the Med-ImageTools index to use for mapping TCIA IDs to local file names
mit_index = loadFileToDataFrame((RAW_DATA_PATH / "images" / config["MIT_INDEX_FILE"]))

# SampleID is local file name
# PatientID is TCIA ID
id_map = mit_index['SampleID']
id_map.index = mit_index["PatientID"]
id_map.drop_duplicates(inplace=True)

In [ ]:
# Map the SampleIDs to the clinical data and add as a column for intersection
clinical_data['SampleID'] = clinical_data['PatientID'].map(id_map)

In [38]:
image_type = "full_original"
pyrad_features = loadFileToDataFrame((RESULTS_DATA_PATH / pyradiomics_settings / f"{image_type}_features.csv"))

In [ ]:
def calc_LASSO(X_train, event_status_train, a_lasso):
    """
    Performs LASSO for feature selection
   
    Parameters
    -----------
    X_train: pd.DataFrame
        A dataframe containing only radiomic features
    event_status_train: pd.DataFrame
        A column containing the event status of patients
    a_lasso: float
        The regularization parameter alpha to be used when initializing the LASSO model
       
    Returns
    ----------
    lasso_passed: list
        All feature names with non zero coefficients
    """
 
    lasso = Lasso(a_lasso)
    lasso.fit(X_train, event_status_train)
    lasso_coef = np.abs(lasso.coef_)
 
    all_rad_feats = X_train.columns.values.tolist()
    lasso_passed = np.array(all_rad_feats)[lasso_coef>0]
 
    return(lasso_passed)